In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/middle-game-11-21-for-pre-process-to-fen/combined_frequency_distribution.csv
/kaggle/input/middle-game-11-21-for-pre-process-to-fen/constellation_difficulty_scores.csv
/kaggle/input/middle-game-11-21-for-pre-process-to-fen/ctp_analysis_df.csv
/kaggle/input/2020-chess-dataset/moves/filtered_games_2020_part_17_moves.csv
/kaggle/input/2020-chess-dataset/moves/filtered_games_2020_part_31_moves.csv
/kaggle/input/2020-chess-dataset/moves/filtered_games_2020_part_15_moves.csv
/kaggle/input/2020-chess-dataset/moves/filtered_games_2020_part_24_moves.csv
/kaggle/input/2020-chess-dataset/moves/filtered_games_2020_part_8_moves.csv
/kaggle/input/2020-chess-dataset/moves/filtered_games_2020_part_27_moves.csv
/kaggle/input/2020-chess-dataset/moves/filtered_games_2020_part_22_moves.csv
/kaggle/input/2020-chess-dataset/moves/filtered_games_2020_part_32_moves.csv
/kaggle/input/2020-chess-dataset/moves/filtered_games_2020_part_33_moves.csv
/kaggle/input/2020-chess-dataset/moves/filtered_gam

In [2]:
import glob

def load_yearly_chess_data(year):
    path_pattern = f'/kaggle/input/{year}-chess-dataset/moves/filtered_games_{year}_part_*_moves.csv'
    file_list = glob.glob(path_pattern)
    dataframes = [pd.read_csv(file) for file in file_list]
    yearly_data = pd.concat(dataframes, ignore_index=True)
    return yearly_data
    
def save_yearly_data(year, chess_df):
    # Save the loaded data for the year with the specified naming convention
    output_filename = f'/kaggle/working/middle_moves_{year}_data.csv'
    chess_df.to_csv(output_filename, index=False)
    print(f"Chess data for {year} saved as {output_filename}.")

# Year to process
year_to_process = 2020

# Load the chess data for the specified year
chess_df = load_yearly_chess_data(year_to_process)

# Save the chess data for the specified year
save_yearly_data(year_to_process, chess_df)

Chess data for 2020 saved as /kaggle/working/middle_moves_2020_data.csv.


In [3]:
# Load the ctp_analysis DataFrame
ctp_analysis_df = pd.read_csv("/kaggle/input/middle-game-11-21-for-pre-process-to-fen/ctp_analysis_df.csv")

# Extract the year as the first four characters of the date_played string
ctp_analysis_df['year_played'] = ctp_analysis_df['date_played'].str[:4].astype(int)

# Filter the DataFrame for the specified year 
ctp_df_for_year = ctp_analysis_df[ctp_analysis_df['year_played'] == year_to_process]

In [4]:
def add_fen_columns(ctp_df, chess_df):
    for index, row in ctp_df.iterrows():
        game_id = row['game_id']
        ply = row['constellation_ply']  # Using the original ply value
        
        matching_rows = chess_df[(chess_df['game_id'] == game_id) & (chess_df['move_no'] == ply)]
        if matching_rows.empty:
            print(f"Skipping game_id {game_id} at ply {ply} - no match found.")
            continue
        
        start_idx = matching_rows.index[0]
        total_moves = len(chess_df)
        # Calculating indexes
        max_constellation_index = min(start_idx + 10, total_moves - 1)
        max_reaction_index = min(start_idx + 30, total_moves - 1)
        
        # Extracting FENs for constellation moves
        for i in range(1, 12):  # 11 moves for the constellation
            fen_col_name = f'constellation_fen_{i}'
            ctp_df.loc[index, fen_col_name] = chess_df['fen'].iloc[start_idx + i - 1] if start_idx + i - 1 <= max_constellation_index else pd.NA
                
        # Extract FENs for subsequent 20 moves
        for i in range(20):  # 20 reaction moves
            fen_col_name = f'reaction_fen_{i+1}'
            # Indexing to start right after the constellation ends
            if start_idx + 11 + i <= max_reaction_index:
                ctp_df.loc[index, fen_col_name] = chess_df['fen'].iloc[start_idx + 11 + i]
            else:
                ctp_df.loc[index, fen_col_name] = pd.NA

                
# Apply FEN columns augmentation for the specific year
add_fen_columns(ctp_df_for_year, chess_df)

# Save the augmented DataFrame directly, no need to concatenate
output_filename = f'/kaggle/working/augmented_ctp_{year_to_process}_data.csv'
ctp_df_for_year.to_csv(output_filename, index=False)
print(f"Augmented CTP data for {year_to_process} saved as {output_filename}.")

/tmp/ipykernel_19/598938033.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ctp_df[f'constellation_fen_{i}'] = None
/tmp/ipykernel_19/598938033.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ctp_df[f'constellation_fen_{i}'] = None
/tmp/ipykernel_19/598938033.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

Augmented CTP data for 2020 saved as /kaggle/working/augmented_ctp_2020_data.csv.
